In [1]:
import pandas as pd
import os
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select

from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import urllib
import pathlib
import datetime


In [2]:
CHROMEDRIVER_PATH = './chromedriver'
WINDOW_SIZE = "1920,1080"

In [3]:
# Comman function for init driver
def init_driver(url):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    
    pathlib.Path('download').mkdir(parents=True, exist_ok=True) 
    
    preferences = {
        "download.default_directory": os.getcwd() + os.path.sep + '/download',
    }

    chrome_options.add_experimental_option('prefs', preferences)

    driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,options=chrome_options)
    driver.maximize_window()
    driver.get(url)
    return driver

In [4]:
def wait_for_page_load(xpath_url):
    try:
        element_present = EC.presence_of_element_located((By.XPATH, xpath_url))
        WebDriverWait(driver, 5).until(element_present)
    except TimeoutException:
        pass
    except NoSuchElementException:
        print("Element not found")

# 1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [5]:
product_name = input('What do you want to search ? ')

What do you want to search ? guitar


In [6]:
driver = init_driver('https://www.amazon.in/')

In [7]:
# add data to keyword field 
driver.find_element_by_xpath('//input[@id="twotabsearchtextbox"]').send_keys(product_name)

# hit search 
driver.find_element_by_xpath('//input[@id="nav-search-submit-button"]').click()

# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [8]:
page_count = 0 
all_urls = []   
while page_count < 3:
    sleep(5)
    # wait for page load
    wait_for_page_load('//div[@class="a-section a-spacing-medium"]')    

    # finding all boxes 
    all_box = driver.find_elements_by_xpath('//div[@class="a-section a-spacing-medium"]')

    
    # Ittrating over box to find details
    for one_box in all_box:

        try:
            url = one_box.find_element_by_xpath('.//h2/a').get_attribute('href')
            all_urls.append(url)
        except:
            pass

        
    # Go to next page
    driver.find_element_by_xpath('//body').send_keys(Keys.CONTROL + Keys.PAGE_DOWN)
    try:
        driver.find_element_by_xpath('//li[@class="a-last"]').click()
    except:
        driver.find_element_by_xpath('//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]').click()
    page_count += 1
    

In [9]:
final_data = []
for one_url in all_urls:
    driver.get(one_url)
    
    try:
        sleep(5)
        wait_for_page_load('//span[@id="priceblock_ourprice"]')    


        try:
            brand_name = driver.find_element_by_xpath('//div[@id="productOverview_feature_div"]//span[@class="a-size-base"]').tex
        except:
            brand_name = driver.find_element_by_xpath('//a[@id="bylineInfo"]').text

        product_name = driver.find_element_by_xpath('//h1[@id="title"]').text
        try:
            rating = driver.find_element_by_xpath('//span[@id="acrPopover"]//span//span').get_attribute('innerHTML')
            ratings_count = driver.find_element_by_xpath('//span[@id="acrCustomerReviewText"]').text
        except:
            rating = '-'
            ratings_count = '-'

        try:
            price = driver.find_element_by_xpath('//span[@id="priceblock_ourprice"]').text
        except:
            try:
                price = driver.find_element_by_xpath('//span[@id="priceblock_dealprice"]').text
            except:
                price = '-'


        return_exchange = driver.find_element_by_xpath('//div[@id="RETURNS_POLICY"]').text
        try:
            expected_delivery = driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b').text
        except:
            expected_delivery = '-'
        availability = driver.find_element_by_xpath('//div[@id="availability_feature_div"]').text
        other_details = driver.find_element_by_xpath('//div[@id="feature-bullets"]').text
        product_url = one_url

        final_data.append({
            'brand_name': brand_name,
            'product_name': product_name,
            'rating': rating,
            'ratings_count': ratings_count,
            'price': price,
            'return_exchange': return_exchange,
            'expected_delivery': expected_delivery,
            'availability': availability,
            'other_details': other_details,
            'product_url': product_url,
        })
    except:
        print('Got error in: ',one_url)
        
data = pd.DataFrame(final_data)
data

,brand_name,product_name,rating,ratings_count,price,return_exchange,expected_delivery,availability,other_details,product_url
0,Visit the Kadence Store,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,3.9 out of 5 stars,"1,028 ratings","₹6,499.00",7 Days Replacement,"Saturday, Oct 2",In stock.,About this item\nQUALITY STRINGS ---The surfac...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Visit the Guitar Bro Store,GUITAR BRO - COMBO (Black Acoustic Guitar for ...,3.7 out of 5 stars,28 ratings,"₹6,499.00",7 Days Replacement,Oct 1 - 6,In stock.,About this item\nGUITAR BRO +20 mins FREE VIDE...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Brand: LME,Laxmi Musical Electronics - Acoustic Pickup fo...,3.7 out of 5 stars,8 ratings,₹599.00,7 Days Replacement,"Thursday, Sep 30",In stock.,Transducer\n› See more product details,https://www.amazon.in/gp/slredirect/picassoRed...
3,Visit the JUAREZ Store,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",4.0 out of 5 stars,"13,094 ratings","₹2,649.00",7 Days Replacement,"Thursday, Sep 30",In stock.,"About this item\nBlack Glossy Finish, Number o...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
4,Visit the JUAREZ Store,"Juarez JRZ38C Acoustic Guitar, 38 Inch Cutaway...",4.0 out of 5 stars,"4,406 ratings","₹2,519.00",7 Days Replacement,"Thursday, Sep 30",In stock.,"About this item\nBlack glossy finish, number o...",https://www.amazon.in/JUAREZ-JRZ38C-Acoustic-S...
...,...,...,...,...,...,...,...,...,...,...
157,Visit the VAULT Store,Vault CL-20 39 Inch Classical Guitar With Bag ...,3.3 out of 5 stars,4 ratings,"₹4,277.00",7 Days Replacement,"Thursday, Sep 30",In stock.,About this item\nAcoustic Classical Nylon Guit...,https://www.amazon.in/Vault-CL-20-Inch-Classic...
158,Visit the GIVSUN Store,Givsun G-39 Black Matt 6-String Cut A Way Righ...,3.4 out of 5 stars,6 ratings,"₹4,099.00",7 Days Replacement,"Thursday, Oct 7",Only 2 left in stock.,About this item\nAcoustic 40 Inch Steel String...,https://www.amazon.in/Givsun-G-39-6-String-Aco...
159,Brand: SOUVENIR,SOUVENIR 360 Degree Guitar Tuner | Digital Cal...,2.7 out of 5 stars,3 ratings,-,7 Days Replacement,"Saturday, Oct 2",Only 2 left in stock.,About this item\nTuning Instrument: Simplicity...,https://www.amazon.in/SOUVENIR-Guitar-Calibrat...
160,Visit the Kadence Store,"Kadence Guitar Acoustica Series, 41” Jumbo Siz...",4.2 out of 5 stars,156 ratings,"₹7,348.00",7 Days Replacement,"Wednesday, Sep 29",In stock.,About this item\nCLASSIC DESIGN ---Elegant inl...,https://www.amazon.in/Kadence-6-strings-Acoust...


# 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [10]:
keywords = ['fruits', 'cars', 'Machine Learning']

In [11]:
def download_image(all_urls):
    for url in all_urls:
        driver.get(url)
        driver.execute_script(f'a = document.createElement("a");a.setAttribute("download","");a.href = "{url}";document.body.appendChild(a);a.click();')

In [13]:
for key in keywords:
    driver.get('https://images.google.com/')
    driver.find_element_by_xpath('//form//input').send_keys(key)
    driver.find_element_by_xpath('//form//input').send_keys(Keys.ENTER)
    
    
    all_src_list = []
    
    while True:
        all_image_box = driver.find_elements_by_xpath('//div[@id="islmp"]//div[@jsname="DeysSe"]/img')
        
        for one_box in all_image_box:
            try:
                one_box.click()
                sleep(1)
                src = driver.find_element_by_xpath('//div[@id="islsp"]//a/img[contains(@src,"https://")]').get_attribute('src')
                all_src_list.append(src)
            except:
                pass

        driver.find_element_by_xpath('//body').send_keys(Keys.CONTROL + Keys.PAGE_DOWN)
        if len(all_src_list) > 100:
            break
        
    download_image(all_src_list)


# 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1 st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [14]:
driver.get('https://www.flipkart.com/')

In [15]:
# close login popup 
try:
    wait_for_page_load('//button[text()="✕"]')
    driver.find_element_by_xpath('//button[text()="✕"]').click()
except:
    pass

In [16]:
driver.find_element_by_xpath('//input[@name="q"]').send_keys('pixel 4A')
driver.find_element_by_xpath('//button[@type="submit"]').click()

In [17]:
wait_for_page_load('//div[contains(@data-id,"MOB")]')    

# finding top all boxes 
all_box = driver.find_elements_by_xpath('//div[contains(@data-id,"MOB")]')

In [18]:
all_urls = [] 
for one_box in all_box:
    all_urls.append(one_box.find_element_by_xpath('.//a').get_attribute('href'))

In [19]:
final_data = []
for one_url in all_urls:
    driver.get(one_url)
    
    try:
        sleep(5)
        
        driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()

        wait_for_page_load('//h1')    


        brand_name = driver.find_element_by_xpath('//h1').text
        smartphone_name = driver.find_element_by_xpath('//h1').text
        colour = driver.find_element_by_xpath('//td[text()="Color"]/parent::*//li').text
        RAM = driver.find_element_by_xpath('//td[text()="RAM"]/parent::*//li').text
        storage_ROM = driver.find_element_by_xpath('//td[text()="Internal Storage"]/parent::*//li').text
        primary_camera = driver.find_element_by_xpath('//td[text()="Primary Camera"]/parent::*//li').text

        try:    
            secondary_camera = driver.find_element_by_xpath('//td[text()="Secondary Camera"]/parent::*//li').text
        except:
            secondary_camera = '-'

        try:    
            display = driver.find_element_by_xpath('//td[text()="Display Type"]/parent::*//li').text
        except:
            display = '-'

        display_size = driver.find_element_by_xpath('//td[text()="Display Size"]/parent::*//li').text

        resolution = driver.find_element_by_xpath('//td[text()="Resolution"]/parent::*//li').text

        try:
            processor = driver.find_element_by_xpath('//td[text()="Processor Type"]/parent::*//li').text
        except:
            processor = '-'

        try:
            processor_cores = driver.find_element_by_xpath('//td[text()="Processor Core"]/parent::*//li').text
        except:
            processor = '-'

        battery_capacity = driver.find_element_by_xpath('//td[text()="Battery Capacity"]/parent::*//li').text

        price = driver.find_element_by_xpath('//div[@class="CEmiEU"]').text
        product_url = one_url






        final_data.append({
            'brand_name':brand_name,
            'smartphone_name':smartphone_name,
            'colour':colour,
            'RAM':RAM,
            'storage_ROM':storage_ROM,
            'primary_camera':primary_camera,
            'secondary_camera':secondary_camera,
            'display_size':display_size,
            'display':display,
            'resolution':resolution,
            'processor':processor,
            'processor_cores':processor_cores,
            'battery_capacity':battery_capacity,
            'price':price,
            'product_url':product_url,
        })
        
    except:
        print('Got error in: ',one_url)
        
data = pd.DataFrame(final_data)
data

,brand_name,smartphone_name,colour,RAM,storage_ROM,primary_camera,secondary_camera,display_size,display,resolution,processor,processor_cores,battery_capacity,price,product_url
0,"Google Pixel 4a (Just Black, 128 GB) (6 GB RAM)","Google Pixel 4a (Just Black, 128 GB) (6 GB RAM)",Just Black,6 GB,128 GB,12.2MP Rear Camera,8MP Front Camera,14.76 cm (5.81 inch),Full HD+ OLED Display,2340 x 1080 Pixels,Qualcomm Snapdragon 730G,Octa Core,3140 mAh,"₹31,999",https://www.flipkart.com/google-pixel-4a-just-...
1,"REDMI 9 Power (Blazing Blue, 64 GB) (4 GB RAM)","REDMI 9 Power (Blazing Blue, 64 GB) (4 GB RAM)",Blazing Blue,4 GB,64 GB,48MP + 8MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),IPS LCD,2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹11,499₹13,99917% off",https://www.flipkart.com/redmi-9-power-blazing...
2,"REDMI 9 Power (Mighty Black, 64 GB) (4 GB RAM)","REDMI 9 Power (Mighty Black, 64 GB) (4 GB RAM)",Mighty Black,4 GB,64 GB,48MP + 8MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),IPS LCD,2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹11,499₹13,99917% off",https://www.flipkart.com/redmi-9-power-mighty-...
3,"REDMI 9 Power (Electric Green, 64 GB) (4 GB RAM)","REDMI 9 Power (Electric Green, 64 GB) (4 GB RAM)",Electric Green,4 GB,64 GB,48MP + 8MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),IPS LCD,2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹11,499₹13,99917% off",https://www.flipkart.com/redmi-9-power-electri...
4,"REDMI 9 Power (Blazing Blue, 128 GB) (6 GB RAM)","REDMI 9 Power (Blazing Blue, 128 GB) (6 GB RAM)",Blazing Blue,6 GB,128 GB,48MP + 8MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),IPS LCD,2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹13,499₹16,99920% off",https://www.flipkart.com/redmi-9-power-blazing...
5,"REDMI 9 Power (Fiery Red, 64 GB) (4 GB RAM)","REDMI 9 Power (Fiery Red, 64 GB) (4 GB RAM)",Fiery Red,4 GB,64 GB,48MP + 8MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),IPS LCD,2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹11,499₹13,99917% off",https://www.flipkart.com/redmi-9-power-fiery-r...
6,"REDMI 9 Power (Mighty Black, 128 GB) (6 GB RAM)","REDMI 9 Power (Mighty Black, 128 GB) (6 GB RAM)",Mighty Black,6 GB,128 GB,48MP + 8MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),IPS LCD,2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹13,499₹16,99920% off",https://www.flipkart.com/redmi-9-power-mighty-...
7,"REDMI 9 Power (Fiery Red, 128 GB) (6 GB RAM)","REDMI 9 Power (Fiery Red, 128 GB) (6 GB RAM)",Fiery Red,6 GB,128 GB,48MP + 8MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),IPS LCD,2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹13,499₹16,99920% off",https://www.flipkart.com/redmi-9-power-fiery-r...
8,"SAMSUNG M31s (Mirage Blue, 128 GB) (8 GB RAM)","SAMSUNG M31s (Mirage Blue, 128 GB) (8 GB RAM)",Mirage Blue,8 GB,128 GB,64MP + 12MP,32MP Front Camera,16.51 cm (6.5 inch),Super AMOLED,1080 x 2400$$pixel,Exynos 9611 octa core processor,Octa Core,6000 mAh,"₹24,998",https://www.flipkart.com/samsung-m31s-mirage-b...
9,"Redmi 9A (Midnight Black, 32 GB) (2 GB RAM)","Redmi 9A (Midnight Black, 32 GB) (2 GB RAM)",Midnight Black,2 GB,32 GB,13MP Rear Camera,-,16.59 cm (6.53 inch),-,720 x 1600$$pixel,-,Octa Core,5000 mAh,"₹7,517",https://www.flipkart.com/redmi-9a-midnight-bla...


# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [20]:
driver.get('https://www.google.com/maps/')

In [21]:
city = input('Enter city: ')

Enter city: banka


In [22]:
driver.find_element_by_xpath('//input[@id="searchboxinput"]').send_keys(city)
driver.find_element_by_xpath('//input[@id="searchboxinput"]').send_keys(Keys.CONTROL + Keys.ENTER)

In [23]:
current_url = driver.current_url

In [24]:
latitude, longitude, _ = current_url.split('@')[1].split('/')[0].split(',')

In [25]:
print('latitude: ',latitude)
print('longitude: ',longitude)

latitude:  25.2646975
longitude:  86.97241


# 6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [26]:
driver.get('https://trak.in/india-startup-funding-investment-2015/')

In [27]:
all_page_length = driver.find_elements_by_xpath('//div[@class="dataTables_length"]//select')

In [28]:
for one_page_length in all_page_length:
    select = Select(one_page_length)
    select.select_by_visible_text('100')

In [29]:
all_table = pd.read_html(driver.page_source)

In [30]:
df = pd.concat(all_table)
df

,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD),Unnamed: 9
0,1.0,01/04/2021,BYJU’S,Edu-tech,Online tutoring,Bengaluru,Innoven Capital,Series F,460000000,NaN
1,2.0,05/04/2021,Meesho,E-commerce,Online reselling platform,Bengaluru,SoftBank Vision Fund 2,Series E,300000000,NaN
2,3.0,14/04/2021,Swiggy,Online Food Delivery,Online Food Delivery,Bengaluru,"Amansa Holdings, Carmignac, Falcon Edge Capita...",Series J,343000000,NaN
3,4.0,07/04/2021,Groww,FinTech,Investment platform,Bengaluru,"MC Global Edtech, B Capital, Baron, others",Series D,83000000,NaN
4,5.0,14/04/2021,Beldara,E-commerce,Global B2B marketplace,Mumbai,Hindustan Media Ventures,Venture,7400000,NaN
...,...,...,...,...,...,...,...,...,...,...
31,32.0,26/07/2018,TheCapitalNet,Fin-Tech,Unified Global Investments Ecosystem,Hyderabad,Lindwall Family Investments LLC (LFI),Private Equity,500000.0,NaN
32,33.0,30/07/2018,Shuttl,Consumer Internet,Bus Aggregation Platform,Gurugram,Amazon Alexa Fund & Dentsu Ventures,Private Equity,11000000.0,NaN
33,34.0,30/07/2018,Cure Fit,Consumer Internet,Health and fitness Platform,Bengaluru,"IDG Ventures, Accel Partners, Kalaari Capital ...",Private Equity,120000000.0,NaN
34,35.0,31/07/2018,Five Star Group,Fin-Tech,Non-Bank Lending Platform For SME,Chennai,"TPG, Norwest Venture Partners, Sequoia Capital...",Private Equity,100000000.0,NaN


In [31]:
df['Date (dd/mm/yyyy)'] = pd.to_datetime(df['Date (dd/mm/yyyy)'],errors='coerce')

In [32]:
df.dtypes

Sr. No.                       float64
Date (dd/mm/yyyy)      datetime64[ns]
Startup Name                   object
Industry / Vertical            object
Sub-Vertical                   object
City / Location                object
Investors' Name                object
Investment Type                object
Amount (In USD)                object
Unnamed: 9                    float64
dtype: object

In [33]:
df

,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD),Unnamed: 9
0,1.0,2021-01-04,BYJU’S,Edu-tech,Online tutoring,Bengaluru,Innoven Capital,Series F,460000000,NaN
1,2.0,2021-05-04,Meesho,E-commerce,Online reselling platform,Bengaluru,SoftBank Vision Fund 2,Series E,300000000,NaN
2,3.0,2021-04-14,Swiggy,Online Food Delivery,Online Food Delivery,Bengaluru,"Amansa Holdings, Carmignac, Falcon Edge Capita...",Series J,343000000,NaN
3,4.0,2021-07-04,Groww,FinTech,Investment platform,Bengaluru,"MC Global Edtech, B Capital, Baron, others",Series D,83000000,NaN
4,5.0,2021-04-14,Beldara,E-commerce,Global B2B marketplace,Mumbai,Hindustan Media Ventures,Venture,7400000,NaN
...,...,...,...,...,...,...,...,...,...,...
31,32.0,2018-07-26,TheCapitalNet,Fin-Tech,Unified Global Investments Ecosystem,Hyderabad,Lindwall Family Investments LLC (LFI),Private Equity,500000.0,NaN
32,33.0,2018-07-30,Shuttl,Consumer Internet,Bus Aggregation Platform,Gurugram,Amazon Alexa Fund & Dentsu Ventures,Private Equity,11000000.0,NaN
33,34.0,2018-07-30,Cure Fit,Consumer Internet,Health and fitness Platform,Bengaluru,"IDG Ventures, Accel Partners, Kalaari Capital ...",Private Equity,120000000.0,NaN
34,35.0,2018-07-31,Five Star Group,Fin-Tech,Non-Bank Lending Platform For SME,Chennai,"TPG, Norwest Venture Partners, Sequoia Capital...",Private Equity,100000000.0,NaN


In [34]:
start_time = datetime.datetime(2020, 7, 1)
end_time = datetime.datetime(2020, 9, 30)

In [35]:
df = df[(start_time < df['Date (dd/mm/yyyy)']) & (df['Date (dd/mm/yyyy)'] < end_time )].sort_values('Date (dd/mm/yyyy)')
df

,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD),Unnamed: 9
3,4.0,2020-07-04,DeHaat,AgriTech,AgriTech,Patna,"Sequoia Capital India, FMO, Omnivore and AgFun...",Series A,12000000,NaN
8,9.0,2020-07-08,FreshToHome,E-commerce,Food Delivery,Bangalore,Ascent Capital,Venture,16200000,NaN
5,6.0,2020-07-09,Melorra,E-commerce,Online Jewelry Store,Bangalore,"Shadow Holdings, Lightbox.",Debt Financing,"upto 8,900,000",NaN
6,7.0,2020-07-09,1mg,E-commerce,Online Pharmacy,Gurgaon,"Gaja Capital, Tata Capital, Partners Group",In Progress,100000000,NaN
9,10.0,2020-07-13,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,500000,NaN
4,5.0,2020-07-13,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,3300000,NaN
8,9.0,2020-07-14,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,200000,NaN
3,4.0,2020-07-14,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,1700000,NaN
0,1.0,2020-07-15,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,1200000000,NaN
7,8.0,2020-07-15,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,292800,NaN


In [36]:
df.reset_index()

,index,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD),Unnamed: 9
0,3,4.0,2020-07-04,DeHaat,AgriTech,AgriTech,Patna,"Sequoia Capital India, FMO, Omnivore and AgFun...",Series A,12000000,NaN
1,8,9.0,2020-07-08,FreshToHome,E-commerce,Food Delivery,Bangalore,Ascent Capital,Venture,16200000,NaN
2,5,6.0,2020-07-09,Melorra,E-commerce,Online Jewelry Store,Bangalore,"Shadow Holdings, Lightbox.",Debt Financing,"upto 8,900,000",NaN
3,6,7.0,2020-07-09,1mg,E-commerce,Online Pharmacy,Gurgaon,"Gaja Capital, Tata Capital, Partners Group",In Progress,100000000,NaN
4,9,10.0,2020-07-13,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,500000,NaN
5,4,5.0,2020-07-13,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,3300000,NaN
6,8,9.0,2020-07-14,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,200000,NaN
7,3,4.0,2020-07-14,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,1700000,NaN
8,0,1.0,2020-07-15,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,1200000000,NaN
9,7,8.0,2020-07-15,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,292800,NaN


# 7. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [37]:
driver.get('https://www.digit.in/top-products/best-gaming-laptops-40.html')

In [38]:
all_laptop_title = driver.find_elements_by_xpath('//div[@class="TopNumbeHeading sticky-footer"]/a')

In [39]:
all_name = []
for one_title in all_laptop_title:
    all_name.append(one_title.text)
    
name_series = pd.Series(all_name)
name_series

<ipython-input-39-e7336e1e8f16>:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  name_series = pd.Series(all_name)


Series([], dtype: float64)

In [40]:
all_specs = driver.find_elements_by_xpath('//div[@class="Spcs-details"]')

In [41]:
all_spec_list = []
for one_spec in all_specs:

    os_name = one_spec.find_element_by_xpath('.//td[text()="OS"]/parent::*/td[3]').text
    display = one_spec.find_element_by_xpath('.//td[text()="Display"]/parent::*/td[3]').text
    processor = one_spec.find_element_by_xpath('.//td[text()="Processor"]/parent::*/td[3]').text
    memory = one_spec.find_element_by_xpath('.//td[text()="Memory"]/parent::*/td[3]').text
    weight = one_spec.find_element_by_xpath('.//td[text()="Weight"]/parent::*/td[3]').text
    dimension = one_spec.find_element_by_xpath('.//td[text()="Dimension"]/parent::*/td[3]').text
    graphics_processor = one_spec.find_element_by_xpath('.//td[text()="Graphics Processor"]/parent::*/td[3]').text
    
    all_spec_list.append({
        'os' : os_name,
        'display' : display,
        'processor' : processor,
        'memory' : memory,
        'weight' : weight,
        'dimension' : dimension,
        'graphics_processor' : graphics_processor,
    })
    
df = pd.DataFrame(all_spec_list)
df

,os,display,processor,memory,weight,dimension,graphics_processor
0,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630
1,Windows 10 Home,"15.6"" (3840 x 2160)",10th Generation Intel® Core™ i9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NA,NA,NA
2,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070
3,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060
4,Windows 10 Pro,"15.6"" (1920 x 1080)",10th Generation Intel® Core™ i5-10300H | 2.50 GHz,1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB
5,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q
6,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650


In [42]:
df['name'] = name_series
df

,os,display,processor,memory,weight,dimension,graphics_processor,name
0,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,NaN
1,Windows 10 Home,"15.6"" (3840 x 2160)",10th Generation Intel® Core™ i9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NA,NA,NA,NaN
2,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,NaN
3,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,NaN
4,Windows 10 Pro,"15.6"" (1920 x 1080)",10th Generation Intel® Core™ i5-10300H | 2.50 GHz,1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB,NaN
5,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,NaN
6,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,NaN


# 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [6]:
driver.get('https://www.forbes.com/billionaires/')

In [7]:
all_list = []

while True:

    all_boxes = driver.find_elements_by_xpath('//div[@class="table-body"]/div/div[contains(@class,"table-row ")]')
    for one_box in all_boxes:

        rank = one_box.find_element_by_xpath('.//div[@class="rank"]').text
        name = one_box.find_element_by_xpath('.//div[@class="personName"]').text
        net_worth = one_box.find_element_by_xpath('.//div[@class="netWorth"]').text
        age = one_box.find_element_by_xpath('.//div[@class="age"]').text
        citizenship = one_box.find_element_by_xpath('.//div[@class="countryOfCitizenship"]').text
        source = one_box.find_element_by_xpath('.//div[@class="source"]').text
        industry = one_box.find_element_by_xpath('.//div[@class="category"]').text

        all_list.append({
            'rank': rank,
            'name': name,
            'net_worth': net_worth,
            'age': age,
            'citizenship': citizenship,
            'source': source,
            'industry': industry,
        })
        
          
    if len(driver.find_elements_by_xpath('//button[contains(@class,"pagination-btn--next pagination-btn--disabled")]')) == 0:
        # we have more pages go to next
        driver.find_element_by_xpath('//button[contains(@class,"pagination-btn--next")]').click()
    else:
        # we are at last page
        break
        
        
df = pd.DataFrame(all_list)
df

,rank,name,net_worth,age,citizenship,source,industry
0,1.,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2.,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3.,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5.,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology
...,...,...,...,...,...,...,...
2750,2674.,Daniel Yong Zhang,$1 B,49,China,e-commerce,Technology
2751,2674.,Zhang Yuqiang,$1 B,65,China,Fiberglass,Manufacturing
2752,2674.,Zhao Meiguang,$1 B,58,China,gold mining,Metals & Mining
2753,2674.,Zhong Naixiong,$1 B,58,China,conglomerate,Diversified


# 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [46]:
driver.get('https://www.youtube.com/watch?v=kTJczUoc26U')

In [47]:
wait_for_page_load('//ytd-comment-thread-renderer')

# Loading all comments
i = 1 
while True:
    i += 500
    driver.execute_script(f"window.scrollTo(0, {i});")
        
    all_box = driver.find_elements_by_xpath('//ytd-comment-thread-renderer')
    
    sleep(1)
    
    if len(all_box) > 500:
        break

In [48]:
all_list = []
for one_box in all_box:
    comment = one_box.find_element_by_xpath('.//ytd-expander[@id="expander"]').text
    upvote = one_box.find_element_by_xpath('.//span[@id="vote-count-middle"]').text
    time = one_box.find_element_by_xpath('.//div[@id="header-author"]//yt-formatted-string/a').text
    
    all_list.append({
        'comment' : comment,
        'upvote' : upvote,
        'time' : time
    })
    
df = pd.DataFrame(all_list)
df

,comment,upvote,time
0,proud of you,130K,2 months ago
1,Очень крутой трек 😀,173,5 hours ago
2,I hear this every day and it would be never bo...,129,1 day ago
3,No ablo ingles pero es la mejor canción que e ...,355,1 day ago
4,Justin's sweet voice and Laroi's voice togethe...,40,7 hours ago
...,...,...,...
514,Nobody will talk about how their voices are a ...,896,1 month ago
515,Next we need justine ft kid,2,23 hours ago
516,B A L D J U S T I N M O M E N T,,2 hours ago
517,"The song What's Catchy, I Fell in Love With It",1,3 hours ago


# 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [49]:
# going to hostel world

In [50]:
driver.get('https://www.hostelworld.com/s?q=London,%20England&country=England&city=London&type=city&id=3')

In [51]:
final_list = []
page_urls = []


while True:
    sleep(5)
    wait_for_page_load('//div[@class="property-card" and @data-v-62f28bda=""]')
    all_boxs = driver.find_elements_by_xpath('//div[@class="property-card" and @data-v-62f28bda=""]')
    for one_box in all_boxs:
        hostel_name = one_box.find_element_by_xpath('.//h2/a').text
        distance_from_city = one_box.find_element_by_xpath('.//a/span').text
        try:
            ratings = one_box.find_element_by_xpath('.//div[contains(@class,"score")]').text
        except:
            ratings = 'No rating'
            
        total_reviews = one_box.find_element_by_xpath('.//div[contains(@class,"reviews")]').text
        overall_reviews = one_box.find_element_by_xpath('.//div[contains(@class,"keyword")]').text
        privates_from_price = one_box.find_elements_by_xpath('.//div[contains(@class,"prices-col")]/a/div')[0].text
        dorms_from_price  = one_box.find_elements_by_xpath('.//div[contains(@class,"prices-col")]/a/div')[1].text
        facilities = one_box.find_element_by_xpath('.//div[contains(@class,"facilities")]').text.split('\n')
        
        page_url = one_box.find_element_by_xpath('.//h2/a').get_attribute('href')
        page_urls.append(page_url)

        final_list.append({
            'hostel_name' : hostel_name,
            'distance_from_city' : distance_from_city,
            'ratings' : ratings,
            'total_reviews' : total_reviews,
            'overall_reviews' : overall_reviews,
            'privates_from_price' : privates_from_price,
            'dorms_from_price' : dorms_from_price,
            'facilities' : facilities,

        })
        
        
    if len(driver.find_elements_by_xpath('//div[contains(@class,"pagination-next disabled")]')) == 0:
        # we have more pages go to next
        driver.find_element_by_xpath('//div[contains(@class,"pagination-next")]').click()
    else:
        # we are at last page
        break
            

df = pd.DataFrame(final_list)
df

,hostel_name,distance_from_city,ratings,total_reviews,overall_reviews,privates_from_price,dorms_from_price,facilities
0,PubLove @ The Rose & Crown,Hostel - 1.6km from city centre,7.9,101 Total Reviews,Very Good,No Privates Available,Dorms From\nRs1719,"[Free WiFi, Follows Covid-19 sanitation guidance]"
1,The Finnish Church in London,Hostel - 4.5km from city centre,10,190 Total Reviews,Superb,No Privates Available,Dorms From\nRs2933,"[Free WiFi, Follows Covid-19 sanitation guidance]"
2,Saint James Backpackers,Hostel - 5.5km from city centre,7.4,1763 Total Reviews,Very Good,Privates From\nRs7849,Dorms From\nRs2528,"[Free WiFi, Free Breakfast, Follows Covid-19 s..."
3,"PubLove @ The Green Man, Paddington",Hostel - 4.3km from city centre,6.2,394 Total Reviews,Good,Privates From\nRs10316,Dorms From\nRs1517,"[Free WiFi, Follows Covid-19 sanitation guidance]"
4,Mornington Camden,Hostel - 4.1km from city centre,10,41 Total Reviews,Superb,No Privates Available,Dorms From\nRs3171,[Free WiFi]
...,...,...,...,...,...,...,...,...
82,Victor Hotel,Hotel - 1.8km from city centre,No rating,2 Total Reviews,No Rating,Privates From\nRs12703,No Dorms Available,[Free WiFi]
83,Best Western Boltons London Kensington,Hotel - 5.4km from city centre,2.0,2 Total Reviews,Rating,Privates From\nRs13593,No Dorms Available,[Free WiFi]
84,Park Hotel Essex,Hotel - 24.1km from city centre,No rating,108 Total Reviews,No Rating,Privates From\nRs4854,No Dorms Available,"[Free Breakfast, Follows Covid-19 sanitation g..."
85,Cranbrook Hotel,Hotel - 14.8km from city centre,No rating,58 Total Reviews,No Rating,Privates From\nRs4854,No Dorms Available,"[Free Breakfast, Follows Covid-19 sanitation g..."


In [52]:
all_desc = []
for one_page in page_urls:
    driver.get(one_page)
    try:
        description = driver.find_element_by_xpath('//div[@class="description-container"]//div[contains(@class,"content")]').text
    except:
        description = ''
        
    all_desc.append(description)    
df_desc = pd.Series(all_desc)
df_desc

0     Food & culture lovers, assemble!\nThe Rose & C...
1     At the The Finnish Church in London, we offer ...
2     Saint James Backpackers, set in a classic Vict...
3     The Green Man Bestplace Inn is next door to Ed...
4     Mornington Camden Hostel is a high class spaci...
                            ...                        
82    The Victor Hotel is a 3-star London hotel set ...
83    Combining Victorian charm with Modern Luxury T...
84    This Hotel is the right choice for visitors wh...
85    We are located about twenty minutes by tube fr...
86    The St Athans Hotel is a proudly simple, famil...
Length: 87, dtype: object

In [53]:
df['description'] = df_desc
df

,hostel_name,distance_from_city,ratings,total_reviews,overall_reviews,privates_from_price,dorms_from_price,facilities,description
0,PubLove @ The Rose & Crown,Hostel - 1.6km from city centre,7.9,101 Total Reviews,Very Good,No Privates Available,Dorms From\nRs1719,"[Free WiFi, Follows Covid-19 sanitation guidance]","Food & culture lovers, assemble!\nThe Rose & C..."
1,The Finnish Church in London,Hostel - 4.5km from city centre,10,190 Total Reviews,Superb,No Privates Available,Dorms From\nRs2933,"[Free WiFi, Follows Covid-19 sanitation guidance]","At the The Finnish Church in London, we offer ..."
2,Saint James Backpackers,Hostel - 5.5km from city centre,7.4,1763 Total Reviews,Very Good,Privates From\nRs7849,Dorms From\nRs2528,"[Free WiFi, Free Breakfast, Follows Covid-19 s...","Saint James Backpackers, set in a classic Vict..."
3,"PubLove @ The Green Man, Paddington",Hostel - 4.3km from city centre,6.2,394 Total Reviews,Good,Privates From\nRs10316,Dorms From\nRs1517,"[Free WiFi, Follows Covid-19 sanitation guidance]",The Green Man Bestplace Inn is next door to Ed...
4,Mornington Camden,Hostel - 4.1km from city centre,10,41 Total Reviews,Superb,No Privates Available,Dorms From\nRs3171,[Free WiFi],Mornington Camden Hostel is a high class spaci...
...,...,...,...,...,...,...,...,...,...
82,Victor Hotel,Hotel - 1.8km from city centre,No rating,2 Total Reviews,No Rating,Privates From\nRs12703,No Dorms Available,[Free WiFi],The Victor Hotel is a 3-star London hotel set ...
83,Best Western Boltons London Kensington,Hotel - 5.4km from city centre,2.0,2 Total Reviews,Rating,Privates From\nRs13593,No Dorms Available,[Free WiFi],Combining Victorian charm with Modern Luxury T...
84,Park Hotel Essex,Hotel - 24.1km from city centre,No rating,108 Total Reviews,No Rating,Privates From\nRs4854,No Dorms Available,"[Free Breakfast, Follows Covid-19 sanitation g...",This Hotel is the right choice for visitors wh...
85,Cranbrook Hotel,Hotel - 14.8km from city centre,No rating,58 Total Reviews,No Rating,Privates From\nRs4854,No Dorms Available,"[Free Breakfast, Follows Covid-19 sanitation g...",We are located about twenty minutes by tube fr...
